In [ ]:
import streamlit as st
import win32com.client
import pythoncom
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from dotenv import load_dotenv

load_dotenv()
st.set_page_config(layout="wide")

# Function to split text into chunks
def split_text_into_chunks(text, max_words=3000, overlap_words=0):
    words = text.split() 
    chunks = []
    for i in range(0, len(words), max_words-overlap_words):
        chunks.append(' '.join(words[i:i + max_words]))
    return chunks

# Function to get key points from text using the Mistral API
def get_key_points(news_text):
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "mistral-tiny"
    client = MistralClient(api_key=api_key)

    # Splitting the text into chunks
    text_chunks = split_text_into_chunks(news_text)
    combined_key_points = []

    # Extracting key points for each chunk
    for chunk in text_chunks:
        chat_response = client.chat(
            model=model,
            messages=[ChatMessage(role="user", content=f"highlight the key points of the following {chunk} ")],
        )
        key_points = chat_response.choices[0].message.content.split('\n')
        combined_key_points.extend(key_points)

    # Optional: Re-process combined points for coherence (This step may require a more complex approach)

    return combined_key_points


# Streamlit app title

# User inputs
email_address = "outlook email address"
# sender_contains = st.text_input("Enter Sender's Email Address or Name:")
# Button to trigger email search and key points extraction
subject_keywords_input ="book, car"  # User inputs keywords
subject_keywords = [keyword.strip() for keyword in subject_keywords_input.split(',')]  # Handle special characters
number_of_emails = 3

# Function to traverse all folders and subfolders
# def traverse_folders(folder, sender_contains, collected_emails):
#     messages = folder.Items
#     for message in messages:
#         try:
#             sender_email = message.SenderEmailAddress.lower()
#             sender_name = message.SenderName.lower()
#             if sender_contains.lower() in sender_email or sender_contains.lower() in sender_name:
#                 collected_emails.append(message)
#         except Exception as e:
#             continue  # Skip any errors and continue with the next message

#     for subfolder in folder.Folders:
#         traverse_folders(subfolder, sender_contains, collected_emails)

# Function to recursively search through folders
def search_folder(folder, subject_keywords, collected_emails, number_of_emails):
    messages = folder.Items
    messages.Sort("[ReceivedTime]", True)  # Sort by received time in descending order

    for message in messages:
        if len(collected_emails) >= number_of_emails:
            break  # Stop if we have collected enough emails

        try:
            subject = message.Subject.lower()
            if any(keyword.lower() in subject for keyword in subject_keywords):
                if message not in collected_emails:  # Ensure uniqueness
                    collected_emails.append(message)
        except Exception as e:
            continue  # Skip any errors

    for subfolder in folder.Folders:
        search_folder(subfolder, subject_keywords, collected_emails, number_of_emails)

# Function to get the Inbox folder and search for emails
def find_emails_and_extract_key_points(account_name, subject_keywords, number_of_emails):
    pythoncom.CoInitialize()  # Initialize the COM library
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    accounts = outlook.Folders

    inbox = None
    for account in accounts:
        if account.Name.lower() == account_name.lower():
            inbox = account.Folders['Inbox']
            break

    if inbox is None:
        st.error("Inbox folder not found for email address: " + account_name)
        return

    collected_emails = []
    search_folder(inbox, subject_keywords, collected_emails, number_of_emails)

    # Extract key points from the collected emails
    for message in collected_emails:
        try:
            key_points = get_key_points(message.Body)
            subject = message.Subject
            st.markdown(f"**Email from {message.SenderName} ({message.SenderEmailAddress}) - Subject: {subject}**")
            for point in key_points:
                updated_point = point.replace("$", "\\$")
                st.markdown(f"<p style='font-family:Century Gothic; font-size:15px; color:black;'>{updated_point}</p>", unsafe_allow_html=True)
        except Exception as e:
            st.error("Error: " + str(e))


if st.button("Extract Key Points from Latest Emails"):
    find_emails_and_extract_key_points(email_address, subject_keywords, number_of_emails)

In [ ]:
import streamlit as st
import win32com.client
import pythoncom
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from dotenv import load_dotenv

load_dotenv()  
# api_key = os.environ["MISTRAL_API_KEY"]
st.set_page_config(layout="wide")

# Function to get key points from text using the Mistral API
def get_key_points(news_text):
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "mistral-tiny"

    client = MistralClient(api_key=api_key)

    chat_response = client.chat(
        model=model,
        messages=[ChatMessage(role="user", content=f"highlight the key points of the following {news_text} ")],
    )

    # Extract bullet points from the response
    key_points = chat_response.choices[0].message.content.split('\n')
    return key_points

# Streamlit app
st.title("Outlook Email Key Points Extractor")

# User inputs
email_address = st.text_input("Enter Your Email Address:")
subfolder_name = st.text_input("Enter the Subfolder Name inside Inbox:")
subject_contains = st.text_input("Enter Text Contained in the Email Subject:")

# Function to get the specified subfolder inside Inbox
def get_subfolder(email_address, subfolder_name):
    pythoncom.CoInitialize()  # Initialize the COM library
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    accounts = outlook.Folders

    for account in accounts:
        if account.Name.lower() == email_address.lower():
            inbox = account.Folders['Inbox']
            try:
                return inbox.Folders[subfolder_name]
            except:
                st.error(f"Subfolder '{subfolder_name}' not found inside Inbox.")
                return None
    return None

# Function to find the email and extract key points
def find_email_and_extract_key_points(folder, subject_contains):
    # Filter emails by subject contains
    messages = folder.Items
    filter_criteria = f"@SQL=urn:schemas:httpmail:subject LIKE '%{subject_contains}%'"
    filtered_messages = messages.Restrict(filter_criteria)

    for message in filtered_messages:
        try:
            # Extract key points
            key_points = get_key_points(message.Body)

            st.markdown(f"**Key Points of the email:**")
            for point in key_points:
                updated_point = point.replace("$", "\\$")
                # st.markdown(f"{updated_point}")
                st.markdown(f"<p style='font-family:Century Gothic; font-size:15px; color:black;'>{updated_point}</p>", unsafe_allow_html=True)
            return
        except Exception as e:
            st.error("Error:" + str(e))
            return
    st.error("No email found with subject containing: " + subject_contains)

# Button to trigger email search and key points extraction
if st.button("Extract Key Points from Email"):
    subfolder = get_subfolder(email_address, subfolder_name)
    if subfolder:
        find_email_and_extract_key_points(subfolder, subject_contains)

In [1]:
import pythoncom

ModuleNotFoundError: No module named 'pythoncom'

In [30]:
import imaplib

OUTLOOK_IMAP_SERVER = 'imap-mail.outlook.com'
OUTLOOK_IMAP_PORT = 993

# Replace these with your email and password
email = 'gopal.goyal02@outlook.com'
password = 'Gopalgoyal@2002'

mail = imaplib.IMAP4_SSL(OUTLOOK_IMAP_SERVER, OUTLOOK_IMAP_PORT)
d=mail.login(email, password)
data=mail.list() # Lists all mailboxes


In [3]:
OUTLOOK_IMAP_SERVER = 'imap-mail.outlook.com'
import imaplib 
import email 
def search_emails(imap_server, email_address, password, sender_contains, number_of_emails):
    mail = imaplib.IMAP4_SSL(imap_server)
    mail.login(email_address, password)
    mail.select('inbox') # Select the Inbox 
    collected_emails = [] # Search for emails from the specific sender 
    status, message_numbers = mail.search(None, '(FROM "{}")'.format(sender_contains)) 
    # print(status,message_numbers)
    if status == 'OK':
        for num in reversed(message_numbers[0].split()): # Iterate in reverse to get the latest emails first 
            if len(collected_emails) >= number_of_emails:
                break # Stop if we have collected enough emails 
            typ, data = mail.fetch(num, '(RFC822)') 
            # print(data)
            for response_part in data: 
                if isinstance(response_part, tuple): 
                    msg = email.message_from_bytes(response_part[1]) 
                    email_subject = decode_header(msg["subject"])[0][0] 
                    if isinstance(email_subject, bytes):
                        email_subject = email_subject.decode() 
                    email_from = decode_header(msg["from"])[0][0] 
                    if isinstance(email_from, bytes): 
                        email_from = email_from.decode() 
                    msg_=(msg.get_payload()[0]).__dict__['_payload']
                    collected_emails.append((email_subject, email_from,msg_ ))
                # print(msg)
    return collected_emails 

In [4]:
search_emails('imap-mail.outlook.com',"shibin.xie@outlook.com","Tsla$7012", "tmurphy@key.com" ,1)

error: b'LOGIN failed. Account is blocked. Login to your account via a web browser to verify your identity.'

In [76]:
!pip install decode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 3.5 MB/s eta 0:00:0000:0100:01m
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.0/970.0 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 2.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.3/248.3 kB 18.3 MB/s eta 0:00:00


In [1]:
import imaplib 
import email 
from email.header import decode_header
 # Function to search emails from a specific sender 
def search_emails(imap_server, email_address, password, sender_contains, number_of_emails):
    mail = imaplib.IMAP4_SSL(imap_server)
    mail.login(email_address, password)
    mail.select('inbox') # Select the Inbox 
    collected_emails = [] # Search for emails from the specific sender 
    status, message_numbers = mail.search(None, '(FROM "{}")'.format(sender_contains)) 
    # print(status,message_numbers)
    if status == 'OK':
        for num in reversed(message_numbers[0].split()): # Iterate in reverse to get the latest emails first 
            if len(collected_emails) >= number_of_emails:
                break # Stop if we have collected enough emails 
            typ, data = mail.fetch(num, '(RFC822)') 
            # print(data)
            for response_part in data: 
                if isinstance(response_part, tuple): 
                    msg = email.message_from_bytes(response_part[1]) 
                    email_subject = decode_header(msg["subject"])[0][0] 
                    if isinstance(email_subject, bytes):
                        email_subject = email_subject.decode() 
                    email_from = decode_header(msg["from"])[0][0] 
                    if isinstance(email_from, bytes): 
                        email_from = email_from.decode() 
                    msg_=(msg.get_payload()[0]).__dict__['_payload']
                    collected_emails.append((email_subject, email_from,msg_ ))
                # print(msg)
    return collected_emails # Replace with your details imap_server = 'imap-mail.outlook.com' email_address = 'your-email@outlook.com' password = 'yourpassword' sender_contains = 'specific sender' number_of_emails = 10 collected_emails = search_emails(imap_server, email_address, password, sender_contains, number_of_emails) # Example: Print the collected emails' subjects and senders for subject, sender, body in collected_emails: print(f"Subject: {subject}, Sender: {sender}") # You can further process the email body or extract key points as neede
def split_text_into_chunks(text, max_words=3000, overlap_words=0):
    words = text.split() 
    chunks = []
    for i in range(0, len(words), max_words-overlap_words):
        chunks.append(' '.join(words[i:i + max_words]))
    return chunks

In [2]:
m = search_emails("imap-mail.outlook.com","gopal.goyal02@outlook.com","Gopalgoyal@2002","gopalgoyal612002@gmail.com",3)

In [3]:
m

[('Python',
  'gopal goyal <gopalgoyal612002@gmail.com>',
  'This is the mail about python codes to learn\r\n'),
 ('Testing', 'gopal goyal <gopalgoyal612002@gmail.com>', 'Testing\r\n')]

In [8]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
def get_key_points(news_text):
    api_key = "HQSYSJjmRHE0dnj9mbnUizFNJLgaNxPG"
    model = "mistral-tiny"
    client = MistralClient(api_key=api_key)

    # Splitting the text into chunks
    text_chunks = split_text_into_chunks(news_text)
    combined_key_points = []

    # Extracting key points for each chunk
    for chunk in text_chunks:
        chat_response = client.chat(
            model=model,
            messages=[ChatMessage(role="user", content=f"highlight the key points of the following {chunk} ")],
        )
        key_points = chat_response.choices[0].message.content.split('\n')
        combined_key_points.extend(key_points)

    # Optional: Re-process combined points for coherence (This step may require a more complex approach)

    return combined_key_points

In [9]:
get_key_points(m[0][1])

["I'm unable to directly view the content of the email you've provided, as it's just mentioned the sender's email address. However, assuming you're asking for a summary of key points from an email written by Gopal Goyal, I'd be happy to help if you can provide a brief description or context of the email's content. Some general key points that could potentially be found in an email from Gopal Goyal might include:",
 '',
 '1. Discussion of business or professional matters, such as project updates, client communications, or strategic planning.',
 '2. Scheduling of meetings or deadlines for completing tasks.',
 '3. Request for information or clarification on specific issues.',
 '4. Sharing of documents or files.',
 '5. Feedback or suggestions for improving processes or collaboration.',
 '6. Personal updates or anecdotes, depending on the nature of the professional relationship.',
 '',
 "Again, this is just a general list based on what emails from a professional contact like Gopal Goyal mig

In [1]:
!pip install mistralai

  Using cached httpx-0.25.2-py3-none-any.whl.metadata (6.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.0/249.0 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.17.3
    Uninstalling httpcore-0.17.3:
      Successfully uninstalled httpcore-0.17.3
  Attempting uninstall: httpx
    Found existing installation: httpx 0.24.1
    Uninstalling httpx-0.24.1:
      Successfully uninstalled httpx-0.24.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
httpx-oauth 0.13.0 requires httpx<0.25,>=0.18, but you have httpx 0.25.2 which is incompatible.


In [5]:
!pip install pytz

In [ ]:
def lambda_handler(event, context):


# Updated API endpoint and authorization token
url = "https://app.nuovopay.com/dm/api/v1/devices.json/?"page="
headers = {
'Authorization': 'Token SECRET_KEy'
}
response = requests.get(url, headers=headers)


data = response.json()


# Write JSON data to S3 bucket
s3 = boto3.resource('s3',
region_name='us-east-1',
aws_access_key_id=access_key,
aws_secret_access_key=secret_key)
s3.Object(bucket_name, 'data.json').put(Body=json.dumps(data))


return {
'statusCode': 200,
'body': json.dumps('success')
}

In [10]:
import imaplib
import pprint


imap_host = 'imap.mail.com'
imap_user = 'username@email.com'
imap_pass = 'password'


# connecting to host via SSL
imap = imaplib.IMAP4_SSL("imap-mail.outlook.com")


In [11]:
print(imap.port)
print("Connected to server.")
print(imap.welcome)



993
Connected to server.
b'* OK The Microsoft Exchange IMAP4 service is ready. [UABOADMAUABSADAAMQBDAEEAMAAwADAAMQAuAEkATgBEAFAAUgBEADAAMQAuAFAAUgBPAEQALgBPAFUAVABMAE8ATwBLAC4AQwBPAE0A]'


In [12]:
# logging in to servers
imap.login("shibin.xie@outlook.com", "Tsla$7012")
print("Login complete.")

# Selecting the inbox of the logged in account
imap.select('Inbox')

error: b'LOGIN failed. Account is blocked. Login to your account via a web browser to verify your identity.'